In [3]:
import torch

# Identity resnet

How to popagate relevance in skip connection

In [4]:
def add_func(a,b):
    return a+b

In [31]:
a = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)
c = add_func(a, b)
c.backward(torch.tensor([10.]))
print(a, a.grad, a*a.grad)
print(b, b.grad, b*b.grad)

tensor([0.4888], requires_grad=True) tensor([10.]) tensor([4.8879], grad_fn=<MulBackward0>)
tensor([0.2713], requires_grad=True) tensor([10.]) tensor([2.7127], grad_fn=<MulBackward0>)


usual gradient propagation is not valid. Sum of relevances not equal to initial relevance 10

In [53]:
class Madd_func(torch.autograd.Function):

    @staticmethod
    def forward(ctx, func, a, b):
        ctx.a = a.clone()
        ctx.b = b.clone()
        ctx.func = func
        return func(a,b)
    @staticmethod
    def backward(ctx, R):
        ctx.a.requires_grad_(True)
        ctx.b.requires_grad_(True)
        with torch.enable_grad():
            Z = ctx.func(ctx.a, ctx.b)
            S = R/Z
#         import ipdb; ipdb.set_trace()
            Z.backward(S)
        return None, ctx.a*ctx.a.grad,  ctx.b*ctx.b.grad

In [54]:
a = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)
c = Madd_func.apply(torch.add, a, b)
c.backward(torch.tensor([10.]))
print(a, a.grad)
print(b, b.grad)

tensor([0.4330], requires_grad=True) tensor([4.1622])
tensor([0.6073], requires_grad=True) tensor([5.8378])


backward step should be overloaded

# Maxpool layers

How to propagete relevance in pool layers

In [49]:
a = torch.rand(1,5,5, requires_grad=True)
b = torch.nn.AdaptiveAvgPool2d((1,1))(a)
print(a)
b.backward(torch.tensor([10.]).view(b.shape))
print(a.grad)
print(a*a.grad)
print((a*a.grad).sum())

tensor([[[0.7801, 0.9829, 0.3185, 0.1373, 0.7157],
         [0.3277, 0.4798, 0.9149, 0.8780, 0.8434],
         [0.4910, 0.6230, 0.7462, 0.8135, 0.6183],
         [0.3260, 0.0618, 0.4656, 0.9856, 0.1830],
         [0.7073, 0.5595, 0.9779, 0.4747, 0.1841]]], requires_grad=True)
tensor([[[0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
         [0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
         [0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
         [0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
         [0.4000, 0.4000, 0.4000, 0.4000, 0.4000]]])
tensor([[[0.3120, 0.3932, 0.1274, 0.0549, 0.2863],
         [0.1311, 0.1919, 0.3659, 0.3512, 0.3374],
         [0.1964, 0.2492, 0.2985, 0.3254, 0.2473],
         [0.1304, 0.0247, 0.1862, 0.3942, 0.0732],
         [0.2829, 0.2238, 0.3912, 0.1899, 0.0736]]], grad_fn=<MulBackward0>)
tensor(5.8383, grad_fn=<SumBackward0>)


In [66]:
class Mavgpool_func(torch.autograd.Function):

    @staticmethod
    def forward(ctx, func, a, args_dict):
        ctx.a = a.clone()
        ctx.func = func
        ctx.args_dict = args_dict
        return func(a, **args_dict)
    @staticmethod
    def backward(ctx, R):
        ctx.a.requires_grad_(True)
        with torch.enable_grad():
            Z = ctx.func(ctx.a, **ctx.args_dict)
            S = R/Z
#         import ipdb; ipdb.set_trace()
            Z.backward(S)
        return None, ctx.a*ctx.a.grad, None

and again backward step should be overloaded

In [71]:
a = torch.rand(1,5,5, requires_grad=True)
b = Mavgpool_func.apply(torch.nn.functional.adaptive_avg_pool2d, a, {'output_size': (1,1)})
print(a)
b.backward(torch.tensor([10.]).view(b.shape))
print(a.grad)
print(a.grad.sum())

tensor([[[0.1457, 0.8833, 0.4814, 0.0671, 0.6818],
         [0.4463, 0.2085, 0.9480, 0.4361, 0.5725],
         [0.7493, 0.1586, 0.4012, 0.1397, 0.8477],
         [0.8180, 0.4504, 0.8729, 0.2440, 0.4761],
         [0.4300, 0.6401, 0.0823, 0.2966, 0.6457]]], requires_grad=True)
tensor([[[0.1202, 0.7286, 0.3971, 0.0553, 0.5624],
         [0.3681, 0.1720, 0.7819, 0.3597, 0.4722],
         [0.6181, 0.1308, 0.3309, 0.1152, 0.6992],
         [0.6747, 0.3715, 0.7200, 0.2013, 0.3927],
         [0.3547, 0.5280, 0.0679, 0.2447, 0.5326]]])
tensor(10.0000)


Now sum of obtained relevances is equal to initial relevance